In [1]:
import random
import numpy as np
#import igraph
import csv
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk

nltk.download('punkt') # for tokenization
nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()

with open("testing_set.txt", "r") as f:
    reader = csv.reader(f)
    testing_set  = list(reader)

testing_set = [element[0].split(" ") for element in testing_set]

[nltk_data] Downloading package punkt to /Users/collinli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/collinli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


IOError: [Errno 2] No such file or directory: 'testing_set.txt'

In [2]:
def inner(X,Y):
    s = 0
    for i in range(0,len(X)):
        s = s + X[i]*Y[i]
    return s

In [3]:
with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)

training_set = [element[0].split(" ") for element in training_set]

with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

IDs = [element[0] for element in node_info]

# compute TFIDF vector of each paper
corpus = [element[5] for element in node_info]
vectorizer = TfidfVectorizer(stop_words="english")
# each row is a node in the order of node_info
features_TFIDF = vectorizer.fit_transform(corpus)

In [33]:
source_tm = features_TFIDF[6].toarray().T
target_tm = features_TFIDF[5].toarray().T
cos_similarity = inner(source_tm,target_tm)\
    / (np.linalg.norm(np.array(source_tm))\
       *np.linalg.norm(np.array(target_tm)))
print(cos_similarity)

[ 0.04330019]


In [35]:
to_keep = random.sample(range(len(training_set)), k=int(round(len(training_set)*0.05)))
training_set_reduced_total = [training_set[i] for i in to_keep]
training_set_reduced = training_set_reduced_total[:len(training_set_reduced_total)/2]
tmp_test = training_set_reduced_total[len(training_set_reduced_total)/2:]

In [129]:
# number of overlapping words in title
overlap_title_tmp_test = []

# temporal distance between the papers
temp_diff_tmp_test = []

# number of common authors
comm_auth_tmp_test = []

text_mining_tmp_test = []

counter = 0
for i in xrange(len(tmp_test)):
    source = tmp_test[i][0]
    target = tmp_test[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
	# convert to lowercase and tokenize
    source_title = source_info[2].lower().split(" ")
	# remove stopwords
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]
    
    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]
    
    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",")
    
    overlap_title_tmp_test.append(len(set(source_title).intersection(set(target_title))))
    temp_diff_tmp_test.append(int(source_info[1]) - int(target_info[1]))
    comm_auth_tmp_test.append(len(set(source_auth).intersection(set(target_auth))))
    
    #add text mining
    source_tm = source_info[6]
    target_tm = target_info[6]
    
    cos_similarity = np.dot(source_tm,target_tm)\
    / (np.linalg.norm(np.array(source_tm))\
       *np.linalg.norm(np.array(target_tm)))
    text_mining_tmp_test.append(cos_similarity)
   
    counter += 1
    if counter % 1000 == True:
        print counter, "tmp test examples processsed"

1 tmp test examples processsed
1001 tmp test examples processsed
2001 tmp test examples processsed
3001 tmp test examples processsed
4001 tmp test examples processsed
5001 tmp test examples processsed
6001 tmp test examples processsed
7001 tmp test examples processsed
8001 tmp test examples processsed
9001 tmp test examples processsed
10001 tmp test examples processsed
11001 tmp test examples processsed
12001 tmp test examples processsed
13001 tmp test examples processsed
14001 tmp test examples processsed
15001 tmp test examples processsed


In [56]:
# number of overlapping words in title
overlap_title = []

# temporal distance between the papers
temp_diff = []

# number of common authors
comm_auth = []

counter = 0
for i in xrange(len(training_set_reduced)):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
	# convert to lowercase and tokenize
    source_title = source_info[2].lower().split(" ")
	# remove stopwords
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]
    
    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]
    
    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",")
    
    overlap_title.append(len(set(source_title).intersection(set(target_title))))
    temp_diff.append(int(source_info[1]) - int(target_info[1]))
    comm_auth.append(len(set(source_auth).intersection(set(target_auth))))
   
    counter += 1
    if counter % 1000 == True:
        print counter, "training examples processsed"

1 training examples processsed
1001 training examples processsed
2001 training examples processsed
3001 training examples processsed
4001 training examples processsed
5001 training examples processsed
6001 training examples processsed
7001 training examples processsed
8001 training examples processsed
9001 training examples processsed
10001 training examples processsed
11001 training examples processsed
12001 training examples processsed
13001 training examples processsed
14001 training examples processsed
15001 training examples processsed


In [98]:
#text mining technique
from collections import defaultdict
import string
wordCount = defaultdict(int)
punctuation = set(string.punctuation)

for d in node_info:    
    r = ''.join([c for c in d[5].lower() if not c in punctuation])
    l = r.split()
    for i in range(1, len(l)):
        if (l[i-1] not in stpwds) and (l[i] not in stpwds):
            w = l[i-1] + ' ' + l[i]
            wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [102]:
words = [x[1] for x in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
  feat = [0]*len(words)
  r = ''.join([c for c in datum[5].lower() if not c in punctuation])
  l = r.split()
  for i in range(1, len(l)):
        w = l[i-1] + ' ' + l[i]
        if w in words:
            feat[wordId[w]] += 1
  feat.append(1) #offset
  return feat

X_train = [feature(d) for d in node_info]

In [111]:
for i in xrange(len(node_info)):
    node_info[i].append(X_train[i])

In [36]:
# number of overlapping words in title
overlap_title_tm = []

# temporal distance between the papers
temp_diff_tm = []

# number of common authors
comm_auth_tm = []

# text_mining vector
text_mining = []

counter = 0
for i in xrange(len(training_set_reduced)):
    source = training_set_reduced[i][0]
    target = training_set_reduced[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    # convert to lowercase and tokenize
    source_title = source_info[2].lower().split(" ")
	# remove stopwords
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]
    
    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]
    
    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",")
    
    overlap_title_tm.append(len(set(source_title).intersection(set(target_title))))
    temp_diff_tm.append(int(source_info[1]) - int(target_info[1]))
    comm_auth_tm.append(len(set(source_auth).intersection(set(target_auth))))
    
    #add text mining
    source_tm = features_TFIDF[index_source].toarray().T
    target_tm = features_TFIDF[index_target].toarray().T
    
    cos_similarity = inner(source_tm,target_tm)\
    / (np.linalg.norm(np.array(source_tm))\
       *np.linalg.norm(np.array(target_tm)))
    text_mining.append(cos_similarity)
    
    counter += 1
    if counter % 1000 == True:
        print counter, "training examples processsed"

1 training examples processsed
1001 training examples processsed
2001 training examples processsed
3001 training examples processsed
4001 training examples processsed
5001 training examples processsed
6001 training examples processsed
7001 training examples processsed
8001 training examples processsed
9001 training examples processsed
10001 training examples processsed
11001 training examples processsed
12001 training examples processsed
13001 training examples processsed
14001 training examples processsed
15001 training examples processsed


In [122]:
len(source_info[6])

27770

In [37]:
# convert list of lists into array
# documents as rows, unique words as columns (i.e., example as rows, features as columns)
#training_features = np.array([overlap_title, temp_diff, comm_auth]).T
training_features = np.array([overlap_title_tm, temp_diff_tm, comm_auth_tm, text_mining]).T

# scale
training_features = preprocessing.scale(training_features)

array([-0.57103718, -0.87475768, -0.23554775, -0.47786327])

In [38]:
# convert labels into integers then into column array
labels = [int(element[2]) for element in training_set_reduced]
labels = list(labels)
labels_array = np.array(labels)

In [82]:
#random prediction
#predictions_random = np.random.choice([0, 1], size=len(tmp_test_features))

In [40]:
#linear regression
theta = np.linalg.lstsq(training_features, labels_array)[0]
print theta

[ 0.0697167   0.11627358  0.00881498  0.22260221]


In [1]:
# initialize basic SVM
classifier = svm.LinearSVC()
# train
classifier.fit(training_features, labels_array)

NameError: name 'svm' is not defined

In [41]:
# test
# we need to compute the features for the testing set

overlap_title_test = []
temp_diff_test = []
comm_auth_test = []
text_mining = []
   
counter = 0
for i in xrange(len(testing_set)):
    source = testing_set[i][0]
    target = testing_set[i][1]
    
    index_source = IDs.index(source)
    index_target = IDs.index(target)
    
    source_info = [element for element in node_info if element[0]==source][0]
    target_info = [element for element in node_info if element[0]==target][0]
    
    source_title = source_info[2].lower().split(" ")
    source_title = [token for token in source_title if token not in stpwds]
    source_title = [stemmer.stem(token) for token in source_title]
    
    target_title = target_info[2].lower().split(" ")
    target_title = [token for token in target_title if token not in stpwds]
    target_title = [stemmer.stem(token) for token in target_title]
    
    source_auth = source_info[3].split(",")
    target_auth = target_info[3].split(",")
    
    overlap_title_test.append(len(set(source_title).intersection(set(target_title))))
    temp_diff_test.append(int(source_info[1]) - int(target_info[1]))
    comm_auth_test.append(len(set(source_auth).intersection(set(target_auth))))
    
    #add text mining
    source_tm = features_TFIDF[index_source].toarray().T
    target_tm = features_TFIDF[index_target].toarray().T
    
    cos_similarity = inner(source_tm,target_tm)\
    / (np.linalg.norm(np.array(source_tm))\
       *np.linalg.norm(np.array(target_tm)))
    text_mining.append(cos_similarity)
   
    counter += 1
    if counter % 1000 == True:
        print counter, "testing examples processsed"
        
# convert list of lists into array
# documents as rows, unique words as columns (i.e., example as rows, features as columns)
#testing_features = np.array([overlap_title_test,temp_diff_test,comm_auth_test]).T
#for text mining
testing_features = np.array([overlap_title_test,temp_diff_test,comm_auth_test,text_mining]).T

# scale
testing_features = preprocessing.scale(testing_features)

1 testing examples processsed
1001 testing examples processsed
2001 testing examples processsed
3001 testing examples processsed
4001 testing examples processsed
5001 testing examples processsed
6001 testing examples processsed
7001 testing examples processsed
8001 testing examples processsed
9001 testing examples processsed
10001 testing examples processsed
11001 testing examples processsed
12001 testing examples processsed
13001 testing examples processsed
14001 testing examples processsed
15001 testing examples processsed
16001 testing examples processsed
17001 testing examples processsed
18001 testing examples processsed
19001 testing examples processsed
20001 testing examples processsed
21001 testing examples processsed
22001 testing examples processsed
23001 testing examples processsed
24001 testing examples processsed
25001 testing examples processsed
26001 testing examples processsed
27001 testing examples processsed
28001 testing examples processsed
29001 testing examples proc

ValueError: setting an array element with a sequence.

In [138]:
#predictions_SVM = list(classifier.predict(testing_features))
predictions = zip(range(len(testing_set)), predictions_SVM)

In [73]:
predictions_linear = []
for i in xrange(len(testing_set)):
    r = inner(theta,testing_features[i])
    if r >= 0:
        predictions_linear.append(1)
    else:
        predictions_linear.append(0)
predictions = zip(range(len(testing_set)), predictions_linear)

In [75]:
predictions_linear = []
for i in xrange(len(testing_set)):
    r = inner(theta,testing_features[i])
    predictions_linear.append(r)
predictions = zip(range(len(testing_set)), predictions_linear)

In [76]:
# write predictions to .csv file suitable for Kaggle (just make sure to add the column names)

with open("improved_predictions.csv","wb") as pred1:
    csv_out = csv.writer(pred1)
    csv_out.writerow(['id','category'])
    for row in predictions:
        csv_out.writerow(row)

In [132]:
tmp_test_features = np.array([overlap_title_tmp_test,temp_diff_tmp_test,comm_auth_tmp_test,text_mining_tmp_test]).T
# scale
tmp_test_features = preprocessing.scale(tmp_test_features)

# convert labels into integers then into column array
tmp_test_labels = [int(element[2]) for element in training_set_reduced]
tmp_test_labels = list(labels)
tmp_test_labels_array = np.array(labels)

In [133]:
predictions_linear = []
for i in xrange(len(tmp_test_labels_array)):
    r = inner(theta,tmp_test_features[i])
    if r >= 0.5:
        predictions_linear.append(1)
    else:
        predictions_linear.append(0)

1


In [60]:
predictions_SVM = np.array(list(classifier.predict(tmp_test_features)))

In [134]:
MSE = 0
for i in xrange(len(predictions_linear)):
    MSE += (predictions_linear[i]-tmp_test_labels_array[i])**2
MSE = MSE*1.0 / len(predictions_linear)
print(MSE)

0.543995321029
